In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("vocab size: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size:  65


In [6]:
stoi = {char: i for i, char in enumerate(chars)}
itos = {i: char for i, char in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # takes a string, returns list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # takes list of integers, returns a string

In [7]:
print(encode("i am navdeep"))
print(decode([47, 1, 39, 51, 1, 52, 39, 60, 42, 43, 43, 54]))
assert decode(encode("i am navdeep")) == "i am navdeep"

[47, 1, 39, 51, 1, 52, 39, 60, 42, 43, 43, 54]
i am navdeep


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)

torch.Size([1115394])


In [9]:
# split dataset
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
torch.manual_seed(1337)

batch_size = 4 # independent sequences that will be processed in parallel
block_size = 8 # maximum context length

def get_batch(split):
    """
    returns a batch (x, y) of batch_size = 4 arrays
    arrays are of size block_size = 8
    """
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # get batch_size=4 random numbers from len(data) - block_size. minus block_size is to prevent overflow
    x = torch.stack([data[i:i+block_size] for i in ix]) # stack batch_size=4 arrays cotaining block_size=8 elements from data starting from index in ix
    y = torch.stack([data[i+1:i+1+block_size] for i in ix]) # stack batch_size=4 arrays cotaining block_size=8 elements from data starting from index+1 in ix

    return x, y

In [11]:
xb, yb = get_batch("train")
print(xb)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f'{context.tolist()} --> {target}')

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
[24] --> 43
[24, 43] --> 58
[24, 43, 58] --> 5
[24, 43, 58, 5] --> 57
[24, 43, 58, 5, 57] --> 1
[24, 43, 58, 5, 57, 1] --> 46
[24, 43, 58, 5, 57, 1, 46] --> 43
[24, 43, 58, 5, 57, 1, 46, 43] --> 39
[44] --> 53
[44, 53] --> 56
[44, 53, 56] --> 1
[44, 53, 56, 1] --> 58
[44, 53, 56, 1, 58] --> 46
[44, 53, 56, 1, 58, 46] --> 39
[44, 53, 56, 1, 58, 46, 39] --> 58
[44, 53, 56, 1, 58, 46, 39, 58] --> 1
[52] --> 58
[52, 58] --> 1
[52, 58, 1] --> 58
[52, 58, 1, 58] --> 46
[52, 58, 1, 58, 46] --> 39
[52, 58, 1, 58, 46, 39] --> 58
[52, 58, 1, 58, 46, 39, 58] --> 1
[52, 58, 1, 58, 46, 39, 58, 1] --> 46
[25] --> 17
[25, 17] --> 27
[25, 17, 27] --> 10
[25, 17, 27, 10]

In [71]:
class BigramLangModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # B, T, C - Batch, Time (one char in a block), Channel (embedding)
        # idx and targets are both (B, T) size inputs

        # logits are predictions/scores of next character and here it is simply the embedding of input
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None

        else:
            # logits and targets need to be reshaped for cross entropy
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # max_new_tokens - number of characters to generate
        # idx - (B, T)
        for _ in range(max_new_tokens):
            # this will return the logits for all the tokens in idx
            logits, loss = self.forward(idx) # (B, T, C)
            # take out the logits for only the last token because we'll generate the next char only on the basis of last char
            logits = logits[:, -1, :] # (B, C)
            # get the probs from logits
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the prob distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # concatenate for output
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            # next character will be found out on the basis of last character till the loop runs
        return idx

In [72]:
torch.manual_seed(1337)

m = BigramLangModel(vocab_size)
logits, loss = m(xb, yb)
print(xb.shape, logits.shape, loss)

# generate
idx=torch.zeros((1, 1), dtype=torch.long) # this is acting as the starting character -> tensor([[0]])
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([4, 8]) torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


### token_embedding_table
- The token_embedding_table is of size (65, 65). For each char in vocab, the table will return an array of 65 integers.
- These integers will work as the probability of being the next character. This is why the 2nd dimension of the table is of size 65.

### cross_entropy
- Cross Entropy expects the logits of 2 dimensions and target to be of 1 dimension. That's why, the logits and targets are being reshaped.
- Logits are converted into (BxT, C) and Targets are converted into (BxT).
- In the example output above dimensions of logits is (32, 65) and of targets is (32).
- Cross entropy will take the first row of logits (logits[0, :]) which contains 65 integers and take the first integer from target   (target[0]) and check the probability at "targetth" element, i.e. logits[0, target[0]]